Objective : Return aggregated result with an ML prediction using BQ ML

In [1]:
###
## helpful resource corner
##

# https://cloud.google.com/bigquery/docs/reference/standard-sql/aggregate_functions

In [2]:
import os
import json

from google.cloud import bigquery
from google.cloud import storage

from botocore.exceptions import ClientError

In [3]:
home = os.path.expanduser('~')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=f"{home}/.config/gcloud/alpine-beacon-336222-c6a1c087e3ef.json"

In [4]:
client = bigquery.Client()
project = client.project
dataset_id = "{}.music".format(project)
dataset = bigquery.Dataset(dataset_id)
print(dataset_id)

alpine-beacon-336222.music


In [5]:
tables = client.list_tables(dataset_id)  # Make an API request.

print("Tables contained in '{}':".format(dataset_id))
for table in tables:
#    if table.table_id == "slices": 
        fqn = "{}.{}.{}".format(table.project, table.dataset_id, table.table_id)
        print(fqn)

Tables contained in 'alpine-beacon-336222.music':
alpine-beacon-336222.music.slices


In [6]:
qry = f"SELECT INFO FROM {fqn}"
query_job = client.query(qry)
results = query_job.result()
for row in results:
    print(row)

Row(([{'generated_on': '2017-12-0308:41:42.057563', 'slice': '54000-54999', 'version': 'v1'}],), {'INFO': 0})
Row(([{'generated_on': '2017-12-0308:41:42.057563', 'slice': '54000-54999', 'version': 'v1'}],), {'INFO': 0})
Row(([{'generated_on': '2017-12-0308:41:42.057563', 'slice': '54000-54999', 'version': 'v1'}],), {'INFO': 0})
Row(([{'generated_on': '2017-12-0308:41:42.057563', 'slice': '54000-54999', 'version': 'v1'}],), {'INFO': 0})
Row(([{'generated_on': '2017-12-0308:41:42.057563', 'slice': '54000-54999', 'version': 'v1'}],), {'INFO': 0})
Row(([{'generated_on': '2017-12-0308:41:42.057563', 'slice': '54000-54999', 'version': 'v1'}],), {'INFO': 0})


In [7]:
# this is kinda cringe but it works and I don't have chatty sql swimming around
with open("sql/dml/count_artists.sql", "r") as the_file:
    the_qRy = ""
    while True:
        line = the_file.readline()
        if line == "":
            break
        else:
            the_qRy = the_qRy + " " + line

In [8]:
query_job3 = client.query(the_qRy)
results3 = query_job3.result()
df = results3.to_dataframe()
df.rename(columns={"f0_":"count"}, inplace = True)

In [9]:
#idx = 0
#for result in results3:
#    idx += 1
#    print(result)
#    if idx > 5:
#        break

In [10]:
df.head(10)

,count,ARTIST_NAME
0,4200,Drake
1,2382,LilWayne
2,2172,KanyeWest
3,1986,KendrickLamar
4,1884,Rihanna
5,1680,TheRollingStones
6,1614,Shakira
7,1608,FloridaGeorgiaLine
8,1608,LukeBryan
9,1566,Pitbull


In [11]:
df.tail(10)

,count,ARTIST_NAME
9322,6,CorrinneMay
9323,6,HallanHashim
9324,6,G.E.M.
9325,6,SteveHoly
9326,6,KentBlazy&GarthBrooks
9327,6,NewSpringWorship
9328,6,DavidZowie
9329,6,Lane8
9330,6,LeYouth
9331,6,AmineEdge&DANCE


In [12]:
with open("sql/models/psn_prdn_model.sql", "r") as the_file:
    model_qRy = ""
    while True:
        line = the_file.readline()
        if line == "":
            break
        else:
            model_qRy = model_qRy + " " + line

print(model_qRy)

 CREATE OR REPLACE MODEL `music.psn_prdn_model`
 OPTIONS
   (model_type='linear_reg',
   input_label_cols=['pos']) AS
 SELECT 
     collaborative,
     TRK.pos,
     TRK.artist_name,
     TRK.track_name,
     TRK.duration_ms,
     TRK.album_name
 FROM (
 SELECT *
     FROM `alpine-beacon-336222.music.slices`,
     UNNEST(playlists) AS PLST
     ), UNNEST(tracks) as TRK;


In [13]:
%%time
model_query_job = client.query(model_qRy)
model_results = model_query_job.result()

CPU times: user 32.8 ms, sys: 8.19 ms, total: 41 ms
Wall time: 1min 12s
